In [1]:
import pandas as pd
from loguru import logger
from pathlib import Path
from tqdm import tqdm
import os
import re
import sys
import json
from mri_data import utils
import subprocess
import nibabel as nib
import numpy as np
from reload_recursive import reload_recursive
# from nilearn import plotting

sys.path.insert(0, "/home/srs-9/Projects/ms_mri/analysis/thalamus/helpers")
import utils

In [2]:
reload_recursive(utils)
import utils

In [3]:
with open("/home/srs-9/Projects/ms_mri/data/subject-sessions-longit.json", 'r') as f:
    subject_sessions = json.load(f)

dataroot = Path("/mnt/h/srs-9/thalamus_project/data")
qc_root = Path("/mnt/h/srs-9/thalamus_project/qc")
all_subjects = [int(subid) for subid in list(subject_sessions.keys())]

screenshot_script = "/home/srs-9/Projects/ms_mri/analysis/thalamus/quality_control/save_screenshot.sh"
image_root = Path("/home/srs-9/Projects/ms_mri/analysis/thalamus/quality_control/images")

segmentations = [
    "choroid.nii.gz",
    "aseg-ventricles.nii.gz",
    "aseg-third_ventricle.nii.gz",
    "aseg-fourth_ventricle.nii.gz",
    "aseg-CSF.nii.gz",
    "peripheral_CSF_CHECK.nii.gz",
    "all_CSF.nii.gz"
]
all_subjects = [int(subid) for subid in list(subject_sessions.keys())]

subject_roots = {
    int(sub): dataroot / f"sub{sub}-{sessions[0]}" for sub, sessions in subject_sessions.items()
}

In [4]:
def get_ref_slice(ref_vol):
    """Returns coordinates [cor, sag, axial]

    Args:
        ref_vol (np.ndarray | tuple[np.ndarray]): 

    Returns:
        _type_: tuple[int]
    """
    if isinstance(ref_vol, np.ndarray):
        ref_x = ref_y = ref_z = ref_vol
    else:
        ref_x, ref_y, ref_z = ref_vol

    check_slice_x = 0
    n_voxels_best = 0
    for i in range(0, ref_x.shape[0]):
        ref_slice_mask = ref_x[i, :, :]
        n_voxels = np.sum(ref_slice_mask > 0)
        if  n_voxels > n_voxels_best:
            n_voxels_best = n_voxels
            check_slice_x = i


    check_slice_y = 0
    n_voxels_best = 0
    for i in range(0, ref_y.shape[1]):
        ref_slice_mask = ref_y[:, i, :]
        n_voxels = np.sum(ref_slice_mask > 0)
        if  n_voxels > n_voxels_best:
            n_voxels_best = n_voxels
            check_slice_y = i


    check_slice_z = 0
    n_voxels_best = 0
    for i in range(0, ref_z.shape[2]):
        ref_slice_mask = ref_z[:, :, i]
        n_voxels = np.sum(ref_slice_mask > 0)
        if  n_voxels > n_voxels_best:
            n_voxels_best = n_voxels
            check_slice_z = i

    return check_slice_x, check_slice_y, check_slice_z


#### Save basic sagittal view

In [ ]:
save_root = image_root / "sagittal_view_basic"
if not save_root.exists():
    os.makedirs(save_root)

failed_subs = []
for sub in all_subjects:
    sub_root = subject_roots[sub]
    # cp_seg = nib.load(sub_root/"choroid.nii.gz").get_fdata()
    # vent_seg = nib.load(sub_root/"aseg-ventricles.nii.gz").get_fdata()
    try:
        fourth_vent_seg = nib.load(sub_root/"aseg-fourth_ventricle.nii.gz").get_fdata()
        # third_vent_seg = nib.load(sub_root/"aseg-third_ventricle.nii.gz").get_fdata()

        coords = get_ref_slice(fourth_vent_seg)
    except Exception:
        failed_subs.append(sub)
        coords = (105, 120, 194)
    coord_dict = {
        "coronal": coords[0],
        "sagittal": coords[1],
        "axial": coords[2]
    }
    anat = sub_root / "t1.nii.gz"
    viewport = "sagittal"

    save_path = save_root / f"sub{sub}-anat_{utils.nifti_name(anat.name)}-{viewport}[{coord_dict[viewport]}].jpg"

    cmd = [
        screenshot_script,
        anat,
        viewport,
        save_path,
        str(coords[0]),
        str(coords[1]),
        str(coords[2])
    ]
    if not save_path.exists():
        print(sub)
        try:
            result = subprocess.run(cmd, check=True, capture_output=True)
        except subprocess.CalledProcessError:
            failed_subs.append(sub)
        if not save_path.exists():
            failed_subs.append(sub)


In [ ]:
save_root = image_root / "hips_thomas"
if not save_root.exists():
    os.makedirs(save_root)

failed_subs = []
for sub in all_subjects:
    sub_root = subject_roots[sub]
    try:
        fourth_vent_seg = nib.load(sub_root/"aseg-fourth_ventricle.nii.gz").get_fdata()
        # third_vent_seg = nib.load(sub_root/"aseg-third_ventricle.nii.gz").get_fdata()

        coords = get_ref_slice(fourth_vent_seg)
    except Exception:
        failed_subs.append(sub)
        coords = (105, 120, 194)
    coord_dict = {
        "coronal": coords[0],
        "sagittal": coords[1],
        "axial": coords[2]
    }
    anat = sub_root / "t1.nii.gz"
    viewport = "sagittal"

    save_path = save_root / f"sub{sub}-anat_{utils.nifti_name(anat.name)}-{viewport}[{coord_dict[viewport]}].jpg"

    cmd = [
        screenshot_script,
        anat,
        viewport,
        save_path,
        str(coords[0]),
        str(coords[1]),
        str(coords[2])
    ]
    if not save_path.exists():
        print(sub)
        try:
            result = subprocess.run(cmd, check=True, capture_output=True)
        except subprocess.CalledProcessError:
            failed_subs.append(sub)
        if not save_path.exists():
            failed_subs.append(sub)


In [81]:
failed_subs_prev = [1011, 1015, 1019, 1189, 1189, 1196, 1196, 1196, 1199]

In [ ]:
0.00833934
1.8

0.903984

In [86]:
sub = 1019
sub_root = subject_roots[sub]
# cp_seg = nib.load(sub_root/"choroid.nii.gz").get_fdata()
# vent_seg = nib.load(sub_root/"aseg-ventricles.nii.gz").get_fdata()
try:
    fourth_vent_seg = nib.load(sub_root/"aseg-fourth_ventricle.nii.gz").get_fdata()
    # third_vent_seg = nib.load(sub_root/"aseg-third_ventricle.nii.gz").get_fdata()

    coords = get_ref_slice(fourth_vent_seg)
except Exception:
    failed_subs.append(sub)
    coords = (105, 120, 194)
coord_dict = {
    "coronal": coords[0],
    "sagittal": coords[1],
    "axial": coords[2]
}
anat = sub_root / "t1.nii.gz"
viewport = "sagittal"

save_path = image_root / f"sub{sub}-anat_{utils.nifti_name(anat.name)}-{viewport}[{coord_dict[viewport]}].jpg"

cmd = [
    screenshot_script,
    anat,
    viewport,
    save_path,
    str(coords[0]),
    str(coords[1]),
    str(coords[2])
]
print(sub)
subprocess.run(cmd, check=True, capture_output=True)

1019


CompletedProcess(args=['/home/srs-9/Projects/ms_mri/analysis/thalamus/quality_control/save_screenshot.sh', PosixPath('/mnt/h/srs-9/thalamus_project/data/sub1019-20190608/t1.nii.gz'), 'sagittal', PosixPath('/home/srs-9/Projects/ms_mri/analysis/thalamus/quality_control/images/sub1019-anat_t1-sagittal[115].jpg'), '118', '115', '111'], returncode=0, stdout=b'/mnt/h/srs-9/thalamus_project/data/sub1019-20190608/t1.nii.gz\n', stderr=b'QStandardPaths: wrong permissions on runtime directory /home/srs-9/.xdg, 0755 instead of 0700\n')

### HIPS-THOMAS

In [40]:
save_root = image_root / "hips_thomas"
if not save_root.exists():
    os.makedirs(save_root)
    
sub = 1003
sub_root = subject_roots[sub]

anat = sub_root / "t1.nii.gz"
hipsthomas_l = sub_root / "thomasfull_L.nii.gz"
hipsthomas_r = sub_root / "thomasfull_R.nii.gz"

ref_vol = nib.load(hipsthomas_l).get_fdata()
x, y, z = get_ref_slice(ref_vol)

# nonzero_slices = []
# best_slice = 0
# ref_z = ref_vol
# n_voxels_best = 0
# for i in range(0, ref_vol.shape[2]):
#    ref_slice_mask = ref_z[:, :, i]
#    n_voxels = np.sum(ref_slice_mask > 0)
   
#    if n_voxels > n_voxels_best:
#       n_voxels_best = n_voxels
#       best_slice = i
      
#    if n_voxels > 0:
#       nonzero_slices.append(i)
      

coord_dict = {
   "coronal": str(x),
   "sagittal": str(y),
   "axial": str(z)
}
anat = sub_root / "t1.nii.gz"

viewport = "axial"

save_path = save_root / f"sub{sub}-anat_{utils.nifti_name(anat.name)}-seg_hipsthomas-{viewport}[{coord_dict[viewport]}].png"

cmd = ["fsleyes", "render", "--outfile", f"\"{str(save_path)}\"", "--size", "1920", "1080", 
       "--scene", "ortho", "-xh" , "-yh", 
       "--voxelLoc", *coord_dict.values(),
       "--hideCursor", "--hideLabels",
       str(anat), "-ot", "volume",
       str(hipsthomas_l), "-ot", "label", "-l", "freesurfercolorlut", "-o",
       str(hipsthomas_r), "-ot", "label", "-l", "freesurfercolorlut", "-o"]

script_file = "script.bash"
with open(script_file, 'w') as f:
   f.write("#!/bin/bash\n")
   f.write(" ".join(cmd))

In [41]:
subprocess.run(["bash", script_file])

CompletedProcess(args=['bash', 'script.bash'], returncode=0)

In [24]:
subprocess.run(cmd)

/home/srs-9/fsl/bin/fsleyes: line 3: import: command not found
/home/srs-9/fsl/bin/fsleyes: line 4: import: command not found
/home/srs-9/fsl/bin/fsleyes: line 6: from: command not found
/home/srs-9/fsl/bin/fsleyes: fsleyes: line 9: syntax error near unexpected token `('
/home/srs-9/fsl/bin/fsleyes: fsleyes: line 9: `    sys.argv[0] = re.sub(r'(-script\.pyw?|\.exe)?$', '', sys.argv[0])'


CompletedProcess(args=['bash', 'fsleyes', 'render', '--outfile', '"/home/srs-9/Projects/ms_mri/analysis/thalamus/quality_control/images/hips_thomas/sub1001-anat_t1-seg_hipsthomas-sagittal[100].png"', '--size', '1920', '1080', '--scene', 'ortho', '-xh', '-yh', '--voxelLoc', '100', '100', '177', '--hideCursor', '--hideLabels', '/mnt/h/srs-9/thalamus_project/data/sub1001-20170215/t1.nii.gz', '-ot', 'volume', '/mnt/h/srs-9/thalamus_project/data/sub1001-20170215/thomasfull_L.nii.gz', '-ot', 'label', '-l', 'freesurfercolorlut', '-o', '/mnt/h/srs-9/thalamus_project/data/sub1001-20170215/thomasfull_R.nii.gz', '-ot', 'label', '-l', 'freesurfercolorlut', '-o'], returncode=2)

In [23]:
check_slice_x = 0
ref = fourth_vent_seg
n_voxels_best = 0
for i in range(0, ref.shape[0]):
    ref_slice_mask = ref[i, :, :]
    n_voxels = np.sum(ref_slice_mask > 0)
    if  n_voxels > n_voxels_best:
        n_voxels_best = n_voxels
        check_slice_x = i


check_slice_y = 0
ref = third_vent_seg
n_voxels_best = 0
for i in range(0, ref.shape[1]):
    ref_slice_mask = ref[:, i, :]
    n_voxels = np.sum(ref_slice_mask > 0)
    if  n_voxels > n_voxels_best:
        n_voxels_best = n_voxels
        check_slice_y = i


check_slice_z = 0
n_voxels_best = 0
for i in range(0, cp_seg.shape[2]):
    cp_slice_mask = cp_seg[:, :, i]
    ref_slice_mask = vent_seg[:, :, i]
    n_voxels = np.sum(ref_slice_mask > 0)
    if  n_voxels > n_voxels_best:
        n_voxels_best = n_voxels
        check_slice_z = i

print(check_slice_x, check_slice_y, check_slice_z)

105 141 194


In [20]:
np.sum(vent_slice_mask > 0)

np.int64(0)